In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from datetime import datetime
import time
import random
from tqdm import tqdm
import datetime
import sys
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# 한글 폰트 깨짐을 위해 

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname='/Library/Fonts/Arial Unicode.ttf').get_name()
plt.rc('font', family = font_name)

# -(음수) 깨짐을 위해
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

# 그래츠 출력시 warning 제거 
import warnings
warnings.filterwarnings('ignore')

In [3]:
df= pd.read_csv('data_211110.csv')

In [4]:
df.head()

,game_num,play_date,game_type,result,game_length,champ_name,spell_d,spell_f,level,creep_score,cs_1m
0,5556841013,2021-11-09,솔랭,승리,31분 3초,티모,점멸,점화,17,208,6.7
1,5556635115,2021-11-09,솔랭,패배,18분 57초,탐 켄치,점멸,순간이동,12,108,5.7
2,5554811542,2021-11-08,자유 5:5 랭크,승리,15분 16초,탐 켄치,점멸,순간이동,11,131,8.6
3,5554761116,2021-11-08,자유 5:5 랭크,다시하기,3분 31초,탐 켄치,점멸,순간이동,3,14,4.0
4,5554653886,2021-11-08,자유 5:5 랭크,패배,28분 37초,릴리아,점멸,순간이동,14,149,5.2


In [5]:
df.shape

(820, 11)

In [43]:
# 게임 날짜와 게임 진행 시간이 object로 되어 있는 걸 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820 entries, 0 to 819
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   game_num     820 non-null    int64         
 1   play_date    820 non-null    datetime64[ns]
 2   game_type    820 non-null    object        
 3   result       820 non-null    object        
 4   game_length  820 non-null    datetime64[ns]
 5   champ_name   820 non-null    object        
 6   spell_d      820 non-null    object        
 7   spell_f      820 non-null    object        
 8   level        820 non-null    int64         
 9   creep_score  820 non-null    int64         
 10  cs_1m        820 non-null    float64       
dtypes: datetime64[ns](2), float64(1), int64(3), object(5)
memory usage: 70.6+ KB


In [32]:
df

,game_num,play_date,game_type,result,game_length,champ_name,spell_d,spell_f,level,creep_score,cs_1m
0,5556841013,2021-11-09,솔랭,승리,31분 3초,티모,점멸,점화,17,208,6.7
1,5556635115,2021-11-09,솔랭,패배,18분 57초,탐 켄치,점멸,순간이동,12,108,5.7
2,5554811542,2021-11-08,자유 5:5 랭크,승리,15분 16초,탐 켄치,점멸,순간이동,11,131,8.6
3,5554761116,2021-11-08,자유 5:5 랭크,다시하기,3분 31초,탐 켄치,점멸,순간이동,3,14,4.0
4,5554653886,2021-11-08,자유 5:5 랭크,패배,28분 37초,릴리아,점멸,순간이동,14,149,5.2
...,...,...,...,...,...,...,...,...,...,...,...
815,5550522923,2021-11-06,자유 5:5 랭크,승리,21분 13초,퀸,점멸,정화,13,131,6.2
816,5550443887,2021-11-06,자유 5:5 랭크,패배,32분 31초,릴리아,점멸,강타,15,148,4.6
817,5548862706,2021-11-06,자유 5:5 랭크,승리,41분 55초,릴리아,점멸,순간이동,18,315,7.5
818,5548736001,2021-11-06,자유 5:5 랭크,승리,28분 17초,탐 켄치,점멸,순간이동,15,127,4.5


In [33]:
for i in range(len(df)):
    df['game_length'] = df['game_length'][i].replace('분', '')
    df['game_length'] = df['game_length'][i].replace('초', '')
    df['game_length'] = df['game_length'][i].replace(' ', ':')

In [34]:
df['game_num'] = df['game_num'].astype('int')
df['level'] = df['level'].astype('int')
df['creep_score'] = df['creep_score'].astype('int')
df['cs_1m'] = df['cs_1m'].astype('float64')
df['play_date'] =  pd.to_datetime(df['play_date'], format='%Y-%m-%d')
df['game_length'] =  pd.to_datetime(df['game_length'], format='%M:%S')

In [37]:
# datetime 형식으로 변환 
df['play_date'] =  pd.to_datetime(df['play_date'])
df['game_length'] =  pd.to_datetime(df['game_length'], format='%M:%S')

In [38]:
df.head()

,game_num,play_date,game_type,result,game_length,champ_name,spell_d,spell_f,level,creep_score,cs_1m
0,5556841013,2021-11-09,솔랭,승리,1900-01-01 00:31:03,티모,점멸,점화,17,208,6.7
1,5556635115,2021-11-09,솔랭,패배,1900-01-01 00:31:03,탐 켄치,점멸,순간이동,12,108,5.7
2,5554811542,2021-11-08,자유 5:5 랭크,승리,1900-01-01 00:31:03,탐 켄치,점멸,순간이동,11,131,8.6
3,5554761116,2021-11-08,자유 5:5 랭크,다시하기,1900-01-01 00:31:03,탐 켄치,점멸,순간이동,3,14,4.0
4,5554653886,2021-11-08,자유 5:5 랭크,패배,1900-01-01 00:31:03,릴리아,점멸,순간이동,14,149,5.2


In [39]:
# datetime으로 변환된 것을 확인 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820 entries, 0 to 819
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   game_num     820 non-null    int64         
 1   play_date    820 non-null    datetime64[ns]
 2   game_type    820 non-null    object        
 3   result       820 non-null    object        
 4   game_length  820 non-null    datetime64[ns]
 5   champ_name   820 non-null    object        
 6   spell_d      820 non-null    object        
 7   spell_f      820 non-null    object        
 8   level        820 non-null    int64         
 9   creep_score  820 non-null    int64         
 10  cs_1m        820 non-null    float64       
dtypes: datetime64[ns](2), float64(1), int64(3), object(5)
memory usage: 70.6+ KB


In [40]:
# 결측치가 없는 것을 확인
df.isnull().sum()

game_num       0
play_date      0
game_type      0
result         0
game_length    0
champ_name     0
spell_d        0
spell_f        0
level          0
creep_score    0
cs_1m          0
dtype: int64

In [12]:
# 정수형 데이터중 이상치 유무 확인
df.describe()

,game_num,level,creep_score,cs_1m
count,7.920000e+02,792.000000,792.000000,792.000000
mean,5.411796e+09,14.232323,143.145202,5.361616
std,8.837231e+07,2.879645,61.478582,1.662585
min,5.251965e+09,2.000000,1.000000,0.100000
25%,5.335379e+09,12.000000,106.000000,4.500000
50%,5.427644e+09,14.000000,142.000000,5.600000
75%,5.487377e+09,16.000000,183.500000,6.500000
max,5.554812e+09,25.000000,361.000000,11.100000


In [42]:
df[df['creep_score'] == 1]

,game_num,play_date,game_type,result,game_length,champ_name,spell_d,spell_f,level,creep_score,cs_1m


In [8]:
df.dtypes

game_num         int64
play_date       object
game_type       object
result          object
game_length     object
champ_name      object
spell_d         object
spell_f         object
level            int64
creep_score      int64
cs_1m          float64
dtype: object